In [8]:
import requests
from icecream import ic
from itertools import chain
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import sleep
import time
import csv
import json

#getting the configurations from the config file
with open('working_config.json', 'r') as f:
    config = json.load(f)
api_key = config["api_key"]

response = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/MindfulPlayer?api_key="+api_key)
response.json()

{'id': 'ASxoirwS1mKe8PylW4RP2MGGhiGc2z1t1CGItx6-2ToQDZk',
 'accountId': 'huZrSbIczSmA-iDIyAmKeTCWqHZj8jY8bZVGvAjFDAwJQg',
 'puuid': 'Tp7YHL2yMmQ5FJRxg5UH70rJaxpXQsbMA8023TfUkggNpTWEAluRzD179iBc8cZSo8ojrj9dPEDNhg',
 'name': 'Mindful Player',
 'profileIconId': 3859,
 'revisionDate': 1633026989000,
 'summonerLevel': 336}

In [9]:
response = requests.get("https://ddragon.leagueoflegends.com/api/versions.json")
current_patch = response.json()[0]
current_patch

'11.19.1'

In [10]:
number_matches = "50"

# get all eu challengers and challenger games

In [11]:
challengers = requests.get("https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key="+api_key)
challengers = challengers.json()

In [ ]:
all_matches = []

with tqdm(total = len(challengers["entries"])) as pbar:
    for challenger in challengers["entries"]:
        #ic(challenger)
        account = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+challenger["summonerName"]+"?api_key="+api_key).json()
        #ic(account)
        #ranked summoners rift id = 420
        try:
            matches = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/"+account["puuid"]+"/ids?queue=420&start=0&count="+number_matches+"&api_key="+api_key).json()
        except:
            pass
        all_matches.append(matches)
        time.sleep(-time.time()%2.41)
        pbar.update(1)
        

  1%|▏         | 4/300 [00:09<11:42,  2.37s/it]

In [ ]:
#unnesting the list and removing duplicates:

all_matches = list(chain.from_iterable(all_matches))
final_matches = set(all_matches)
len(final_matches)

In [ ]:
with open('data/challenger_eu_matches.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(final_matches)

In [ ]:
#next up i will read the match data and 

In [9]:
read_matches = []
with open('data/challenger_eu_matches.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        read_matches.append(row)
matches = read_matches[0]
len(matches)

5645

In [10]:
dataset = []
with tqdm(total = len(matches)) as pbar:
    for match in matches:
        match_data = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+match+"?api_key="+api_key).json()
        try:
            if match_data["info"]["gameVersion"][:5] == current_patch[:5]:
                champs = []
                for player in range(10):
                    champs.append(match_data["info"]["participants"][player]["championId"])
                champs.append(int(match_data["info"]["participants"][0]["win"]))
                dataset.append(champs)
                #sleep(1.2)
                time.sleep(-time.time()%1.21)
                pbar.update(1)
            else:
                time.sleep(-time.time()%1.21)
                pbar.update(1)
        except:
            time.sleep(-time.time()%1.21)
            pbar.update(1)

100%|█████████▉| 5642/5645 [1:53:53<00:03,  1.21s/it]


In [11]:
dataset[:5]

[[24, 102, 245, 166, 89, 80, 120, 777, 145, 201, 0],
 [75, 234, 45, 18, 12, 131, 39, 91, 429, 78, 1],
 [2, 59, 147, 67, 412, 114, 234, 13, 10, 555, 1],
 [58, 203, 134, 235, 223, 79, 234, 142, 429, 111, 1],
 [85, 238, 2, 115, 111, 31, 79, 157, 222, 89, 1]]

In [12]:
with open('data/challenger_eu_data.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(dataset)

In [13]:
#a = np.array(dataset)
np.savetxt("data/challenger_eu_data.csv", dataset, delimiter=", ")

# get challengers in korea

In [14]:
challengers = requests.get("https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key="+api_key)
challengers = challengers.json()
challengers["tier"]

'CHALLENGER'

In [15]:
all_matches = []

with tqdm(total = len(challengers["entries"])) as pbar:
    for challenger in challengers["entries"]:
        #ic(challenger)
        account = requests.get("https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+challenger["summonerName"]+"?api_key="+api_key).json()
        #ic(account["puuid"])
        #ranked summoners rift id = 420
        try:
            matches = requests.get("https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/"+account["puuid"]+"/ids?queue=420&start=0&count="+number_matches+"&api_key="+api_key).json()
            #all_matches.append(matches)
        except:
            pass
        all_matches.append(matches)
        time.sleep(-time.time()%2.41)
        pbar.update(1)
        

100%|██████████| 300/300 [22:38<00:00,  4.53s/it]


In [16]:
#unnesting the list and removing duplicates:

all_matches = list(chain.from_iterable(all_matches))
final_matches = set(all_matches)
len(final_matches)

5446

In [17]:
with open('data/challenger_kr_matches.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(final_matches)

In [18]:
#reading the match data and looking at the matches
read_matches = []
with open('data/challenger_kr_matches.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        read_matches.append(row)
matches = read_matches[0]
len(matches)

5446

In [19]:
dataset = []
with tqdm(total = len(matches)) as pbar:
    for match in matches:
        match_data = requests.get("https://asia.api.riotgames.com/lol/match/v5/matches/"+match+"?api_key="+api_key).json()
        try:
            if match_data["info"]["gameVersion"][:5] == current_patch[:5]:
                champs = []
                for player in range(10):
                    champs.append(match_data["info"]["participants"][player]["championId"])
                champs.append(int(match_data["info"]["participants"][0]["win"]))
                dataset.append(champs)
                #sleep(1.2)
                time.sleep(-time.time()%1.21)
                pbar.update(1)
            else:
                time.sleep(-time.time()%1.21)
                pbar.update(1)
        except:
            time.sleep(-time.time()%1.21)
            pbar.update(1)

100%|█████████▉| 5435/5446 [3:13:10<00:23,  2.13s/it]  


In [20]:
dataset[:5]

[[41, 76, 238, 523, 412, 126, 78, 103, 145, 497, 0],
 [24, 131, 64, 523, 350, 150, 234, 69, 119, 89, 0],
 [126, 234, 68, 145, 497, 875, 64, 517, 81, 12, 0],
 [24, 876, 58, 22, 223, 164, 64, 3, 110, 526, 0],
 [154, 68, 236, 523, 412, 164, 234, 142, 115, 223, 0]]

In [21]:
len(dataset)

2507

In [22]:
np.savetxt("data/challenger_kr_data.csv", dataset, delimiter=", ")

# get all eu grandmasters and grandmaster games

In [23]:
gms = requests.get("https://euw1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key="+api_key)
gms = gms.json()
all_matches = []

with tqdm(total = len(gms["entries"])) as pbar:
    for gm in gms["entries"]:
        #ic(challenger)
        account = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+gm["summonerName"]+"?api_key="+api_key).json()
        #ic(account)
        #ranked summoners rift id = 420
        try:
            matches = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/"+gm["puuid"]+"/ids?queue=420&start=0&count="+number_matches+"&api_key="+api_key).json()
        except:
            pass
        all_matches.append(matches)
        time.sleep(-time.time()%2.41)
        pbar.update(1)
        
        
#unnesting the list and removing duplicates:

all_matches = list(chain.from_iterable(all_matches))
final_matches = set(all_matches)
len(final_matches)
with open('data/grandmaster_eu_matches.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(final_matches)

100%|██████████| 700/700 [28:05<00:00,  2.41s/it]


In [4]:
read_matches = []
with open('data/grandmaster_eu_matches.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        read_matches.append(row)
matches = read_matches[0]
len(matches)

5446

In [5]:
dataset = []
with tqdm(total = len(matches)) as pbar:
    for match in matches:
        try:
            match_data = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+match+"?api_key="+api_key).json()
            if match_data["info"]["gameVersion"][:5] == current_patch[:5]:
                champs = []
                for player in range(10):
                    champs.append(match_data["info"]["participants"][player]["championId"])
                champs.append(int(match_data["info"]["participants"][0]["win"]))
                dataset.append(champs)
                #sleep(1.2)
                time.sleep(-time.time()%1.21)
                pbar.update(1)
            else:
                time.sleep(-time.time()%1.21)
                pbar.update(1)
        except:
            time.sleep(-time.time()%1.21)
            pbar.update(1)
            
np.savetxt("data/grandmaster_eu_data.csv", dataset, delimiter=", ")

100%|██████████| 5446/5446 [1:49:50<00:00,  1.21s/it]


In [11]:
with open('data/grandmaster_eu_data.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(dataset)

In [12]:
np.savetxt("data/grandmaster_eu_data.csv", dataset, delimiter=", ")

# getting all grandmaster matches in korea

In [6]:
gms = requests.get("https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key="+api_key)
gms = gms.json()
all_matches = []

with tqdm(total = len(gms["entries"])) as pbar:
    for gm in gms["entries"]:
        #ic(challenger)
        account = requests.get("https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+gm["summonerName"]+"?api_key="+api_key).json()
        #ic(account)
        #ranked summoners rift id = 420
        try:
            matches = requests.get("https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/"+gm["puuid"]+"/ids?queue=420&start=0&count="+number_matches+"&api_key="+api_key).json()
        except:
            pass
        all_matches.append(matches)
        time.sleep(-time.time()%2.41)
        pbar.update(1)
        
#unnesting the list and removing duplicates:

all_matches = list(chain.from_iterable(all_matches))
final_matches = set(all_matches)
len(final_matches)
with open('data/grandmaster_kr_matches.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(final_matches)

100%|██████████| 700/700 [28:14<00:00,  2.42s/it]


In [7]:
read_matches = []
with open('data/grandmaster_kr_matches.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        read_matches.append(row)
matches = read_matches[0]
len(matches)

5446

In [8]:
dataset = []
with tqdm(total = len(matches)) as pbar:
    for match in matches:
        match_data = requests.get("https://asia.api.riotgames.com/lol/match/v5/matches/"+match+"?api_key="+api_key).json()
        try:
            if match_data["info"]["gameVersion"][:5] == current_patch[:5]:
                champs = []
                for player in range(10):
                    champs.append(match_data["info"]["participants"][player]["championId"])
                champs.append(int(match_data["info"]["participants"][0]["win"]))
                dataset.append(champs)
                #sleep(1.2)
                time.sleep(-time.time()%1.21)
                pbar.update(1)
            else:
                time.sleep(-time.time()%1.21)
                pbar.update(1)
        except:
            time.sleep(-time.time()%1.21)
            pbar.update(1)
            
np.savetxt("data/grandmaster_kr_data.csv", dataset, delimiter=", ")

 18%|█▊        | 973/5446 [34:26<2:38:20,  2.12s/it]


KeyboardInterrupt: 

# masters in eu

In [ ]:
masters = requests.get("https://euw1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key="+api_key)
masters = masters.json()
all_matches = []

with tqdm(total = len(masters["entries"])) as pbar:
    for master in masters["entries"]:
        #ic(challenger)
        account = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+master["summonerName"]+"?api_key="+api_key).json()
        #ic(account)
        #ranked summoners rift id = 420
        try:
            matches = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/"+master["puuid"]+"/ids?queue=420&start=0&count="+number_matches+"&api_key="+api_key).json()
        except:
            pass
        all_matches.append(matches)
        time.sleep(-time.time()%2.41)
        pbar.update(1)
        
#unnesting the list and removing duplicates:

all_matches = list(chain.from_iterable(all_matches))
final_matches = set(all_matches)
len(final_matches)
with open('data/master_eu_matches.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(final_matches)

In [ ]:
read_matches = []
with open('data/master_eu_matches.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        read_matches.append(row)
matches = read_matches[0]
len(matches)

In [ ]:
dataset = []
with tqdm(total = len(matches)) as pbar:
    for match in matches:
        try:
            match_data = requests.get("https://europe.api.riotgames.com/lol/match/v5/matches/"+match+"?api_key="+api_key).json()
            if match_data["info"]["gameVersion"][:5] == current_patch[:5]:
                champs = []
                for player in range(10):
                    champs.append(match_data["info"]["participants"][player]["championId"])
                champs.append(int(match_data["info"]["participants"][0]["win"]))
                dataset.append(champs)
                #sleep(1.2)
                time.sleep(-time.time()%1.21)
                pbar.update(1)
            else:
                time.sleep(-time.time()%1.21)
                pbar.update(1)
        except:
            time.sleep(-time.time()%1.21)
            pbar.update(1)
            
np.savetxt("data/master_eu_data.csv", dataset, delimiter=", ")

# master KR

In [ ]:
masters = requests.get("https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key="+api_key)
masters = masters.json()
all_matches = []

with tqdm(total = len(masters["entries"])) as pbar:
    for master in masters["entries"]:
        #ic(challenger)
        account = requests.get("https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+master["summonerName"]+"?api_key="+api_key).json()
        #ic(account)
        #ranked summoners rift id = 420
        try:
            matches = requests.get("https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/"+master["puuid"]+"/ids?queue=420&start=0&count="+number_matches+"&api_key="+api_key).json()
        except:
            pass
        all_matches.append(matches)
        time.sleep(-time.time()%2.41)
        pbar.update(1)
        
#unnesting the list and removing duplicates:

all_matches = list(chain.from_iterable(all_matches))
final_matches = set(all_matches)
len(final_matches)
with open('data/master_kr_matches.csv', 'w') as f:
    write = csv.writer(f) 
    write.writerow(final_matches)

In [ ]:
read_matches = []
with open('data/master_kr_matches.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    for row in csv_reader:
        read_matches.append(row)
matches = read_matches[0]
len(matches)

In [ ]:
dataset = []
with tqdm(total = len(matches)) as pbar:
    for match in matches:
        try:
            match_data = requests.get("https://asia.api.riotgames.com/lol/match/v5/matches/"+match+"?api_key="+api_key).json()
            if match_data["info"]["gameVersion"][:5] == current_patch[:5]:
                champs = []
                for player in range(10):
                    champs.append(match_data["info"]["participants"][player]["championId"])
                champs.append(int(match_data["info"]["participants"][0]["win"]))
                dataset.append(champs)
                #sleep(1.2)
                time.sleep(-time.time()%1.21)
                pbar.update(1)
            else:
                time.sleep(-time.time()%1.21)
                pbar.update(1)
        except:
            time.sleep(-time.time()%1.21)
            pbar.update(1)
            
np.savetxt("data/master_eu_data.csv", dataset, delimiter=", ")

# displaying the amount of matches:

In [2]:
call_eu = pd.read_csv('data/challenger_eu_data.csv',header = None)
call_kr = pd.read_csv('data/challenger_kr_data.csv',header = None)
gm_eu = pd.read_csv('data/grandmaster_eu_data.csv',header = None)
#gm_kr = pd.read_csv('data/grandmaster_kr_data.csv',header = None)
#master_eu = pd.read_csv('data/master_eu_data.csv',header = None)
#master_kr = pd.read_csv('data/master_kr_data.csv',header = None)

In [4]:
call_eu

,0,1,2,3,4,5,6,7,8,9,10
0,24.0,102.0,245.0,166.0,89.0,80.0,120.0,777.0,145.0,201.0,0.0
1,75.0,234.0,45.0,18.0,12.0,131.0,39.0,91.0,429.0,78.0,1.0
2,2.0,59.0,147.0,67.0,412.0,114.0,234.0,13.0,10.0,555.0,1.0
3,58.0,203.0,134.0,235.0,223.0,79.0,234.0,142.0,429.0,111.0,1.0
4,85.0,238.0,2.0,115.0,111.0,31.0,79.0,157.0,222.0,89.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2252,58.0,131.0,202.0,234.0,101.0,39.0,64.0,7.0,67.0,412.0,1.0
2253,164.0,106.0,13.0,119.0,48.0,54.0,64.0,127.0,110.0,526.0,0.0
2254,23.0,106.0,134.0,360.0,12.0,92.0,76.0,166.0,523.0,25.0,0.0
2255,14.0,234.0,7.0,18.0,79.0,58.0,141.0,157.0,517.0,117.0,0.0


In [3]:
print(f"EU challengers = {len(call_eu)}")
print(f"KR challengers = {len(call_kr)}")
print(f"EU gms = {len(gm_eu)}")
print(f"KR gms = {len(gm_kr)}")
print(f"EU masters = {len(master_eu)}")
print(f"KR masters = {len(master_kr)}")
print(f"all games = {len(call_eu)+len(call_kr)+len(gm_eu)+len(gm_kr)+len(master_eu)+len(master_kr)}")

EU challengers = 2257
KR challengers = 2507
EU gms = 432


NameError: name 'gm_kr' is not defined

# d1 if possible, i will not use d1 games